# Building a Dataset of Features

Now that I know which features I would like to extract to build a model, I will compile them in a single matrix. 

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import librosa
import librosa.display

## Firearm Audio Files for Training


In [6]:
firearm_path='/Users/aarojas/Documents/Data_Science_Resources/Insight_20B/AAR_Insight_Project/Firearm_Detection/data/raw/Firearms/'
firearm_audio=[]
for file in os.listdir(firearm_path):
    if file.endswith('.wav'):
        firearm_audio.append(file)
    if file.endswith('.mp3'):
        firearm_audio.append(file)
    if file.endswith('.aiff'):
        firearm_audio.append(file)
print('The number of audio files are:',len(firearm_audio))
print('The number of audio files used for training are ',len(firearm_audio[0:int(46*.7)]))
import pprint
print('\n')
print('The number of audio files used for testing are ',len(firearm_audio[int(46*.7):]))
pprint.pprint(firearm_audio[int(46*.7):])
firearm_test_files=firearm_audio[int(46*.7):]

The number of audio files are: 46
The number of audio files used for training are  32


The number of audio files used for testing are  14
['217805__gattoangus__9mm-pistol-shoot-short-reverb.wav',
 '329716__812402__gun-shot.mp3',
 '344142__brokenphono__gunshot-002.wav',
 '459116__mabdurrahman__gun-shot-1.wav',
 '35799__kante__glock-one-shot.wav',
 'GUN-RICOCHET_GEN-HDF-13674.wav',
 '157800__soundcollectah__gshot-02.mp3',
 'GUN-BULLET_GEN-HDF-13248.wav',
 'Colt 1911 gunfire sound effect.mp3',
 '84254__tito-lahaye__45-smith-wesson.wav',
 '385785__rijam__rifle-shot-m1-garand.mp3',
 '467403__sergeyionov__mng-firearm-shot.wav',
 '182271__martian__gun-for-lui.wav',
 '9mm Gunshot Sound Effect!.mp3']


## Environment Audio Files for Training
I'm going to reduce this dataset by one fourth--to make things run faster for now. I will keep the first half of audio files in each category.

In [12]:
esc50audio_path='/Users/aarojas/Documents/Data_Science_Resources/Insight_20B/AAR_Insight_Project/Firearm_Detection/data/raw/ESC-50-master/audio/'

esc50csv_path='/Users/aarojas/Documents/Data_Science_Resources/Insight_20B/AAR_Insight_Project/Firearm_Detection/data/raw/ESC-50-master/meta/'
esc50=pd.read_csv(esc50csv_path+'esc50.csv',sep=',')

esc50_reduced=[]
esc50_test_files=[]
esc50bycategory=esc50.groupby(['category'])

for name, group in esc50bycategory:
    esc50_reduced+=list(group['filename'].iloc[0:10])
    esc50_test_files.append(group['filename'].iloc[25])

print(len(esc50_reduced))
print(esc50_reduced[1:5])
print(esc50_test_files[:3])


500
['1-24796-A-47.wav', '1-36929-A-47.wav', '1-43807-A-47.wav', '1-43807-B-47.wav']
['4-161099-B-47.wav', '4-176914-A-23.wav', '4-144468-B-27.wav']


## Helper Function to Obtain Features from Librosa

In [14]:
def obtainFeatures(audio_path,file,printtxt=False):
    '''
    audio_path is a string to the path of an audio file
    filenames is a list of strings of the names of the files''' 

    y,sr=librosa.load(audio_path+file,duration=5)

    mfcc=librosa.feature.mfcc(y)

    # the log Mel Spectrogram
    melspec=librosa.feature.melspectrogram(y)
    log_melspec=librosa.amplitude_to_db(melspec)

    # the HPSS
    y_harmonic,y_percussive=librosa.effects.hpss(y)
    melspec=librosa.feature.melspectrogram(y_harmonic)
    log_harmonic=librosa.amplitude_to_db(melspec)
    melspec=librosa.feature.melspectrogram(y_percussive)
    log_percussive=librosa.amplitude_to_db(melspec)
    
    if printtxt==True:
        print('The shape of mfcc is: ',mfcc.shape)
        print('The shape of log_melspec is :',log_melspec.shape)
        print('The shape of log_percussive is', log_percussive.shape)
        print('The shape of log_harmonic is', log_harmonic.shape)
        print('\n')
    return mfcc,log_melspec,log_harmonic,log_percussive

mfcc,log_melspec,log_harm,log_per=obtainFeatures(esc50audio_path,esc50_reduced[0],printtxt=True)

The shape of mfcc is:  (20, 216)
The shape of log_melspec is : (128, 216)
The shape of log_percussive is (128, 216)
The shape of log_harmonic is (128, 216)




Flatten the matrices of interest

In [15]:
def flattenArray(feature,printtxt=False):
    '''feature is a np.array'''
    feature_flat=feature.reshape((1,len(feature.flatten().T)))
    if printtxt==True:
        print('The shape of the feature is ',feature_flat.shape)
    return feature_flat

mfcc_flat=flattenArray(mfcc,printtxt=True)
log_melspec_flat=flattenArray(log_melspec,printtxt=True)
log_harm_flat=flattenArray(log_harm,printtxt=True)
log_per_flat=flattenArray(log_per,printtxt=True)

The shape of the feature is  (1, 4320)
The shape of the feature is  (1, 27648)
The shape of the feature is  (1, 27648)
The shape of the feature is  (1, 27648)


"Center" the data by substracting the mean and dividing by the standard deviation. 

In [16]:
def centerFeature(feature_flat):
    '''feature_flat is a np.array row vector (1,n)'''
    mean=np.mean(feature_flat)
    std=np.std(feature_flat)
    return (feature_flat-mean)/std

MFCC=centerFeature(mfcc_flat)
LOG_MS=centerFeature(log_melspec_flat)
LOG_H=centerFeature(log_harm_flat)
LOG_P=centerFeature(log_per_flat)

In [17]:
processed_data_path='/Users/aarojas/Documents/Data_Science_Resources/Insight_20B/AAR_Insight_Project/Firearm_Detection/data/processed/'
os.chdir(path=processed_data_path)


## Build the Environmental Sound Dataset
First, I will focus on looking at only the mfcc to build a model. 

In [7]:
for file in esc50_reduced[1:]:
    mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(esc50audio_path,file)
    
    mfcc_centered=centerFeature(flattenArray(mfcc))
    log_ms_centered=centerFeature(flattenArray(log_melspec))
    log_h_centered=centerFeature(flattenArray(log_harmonic))
    log_p_centered=centerFeature(flattenArray(log_percussive))
    
    MFCC=np.concatenate((MFCC,mfcc_centered),axis=0)
    LOG_MS=np.concatenate((LOG_MS,log_ms_centered),axis=0)
    LOG_H=np.concatenate((LOG_H,log_h_centered),axis=0)
    LOG_P=np.concatenate((LOG_P,log_p_centered),axis=0)
                         

In [8]:
print(MFCC.shape)
print(LOG_MS.shape)
print(LOG_H.shape)
print(LOG_P.shape)

(500, 4320)
(500, 27648)
(500, 27648)
(500, 27648)


In [9]:

np.savetxt("ESC50_MFCC_train.csv", MFCC, delimiter=",")
np.savetxt("ESC50_logMS_train.csv",LOG_MS,delimiter=",")
np.savetxt("ESC50_logH_train.csv",LOG_H,delimiter=",")
np.savetxt("ESC50_logP_train.csv",LOG_P,delimiter=",")

## Build the Firearms Sound Dataset

In [21]:
mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(firearm_path,firearm_audio[0])
MFCC_fa=centerFeature(flattenArray(mfcc,printtxt=True))
LOG_MS_fa=centerFeature(flattenArray(log_melspec))
LOG_H_fa=centerFeature(flattenArray(log_harmonic))
LOG_P_fa=centerFeature(flattenArray(log_percussive))

for file in firearm_audio[1:32]:
    print(file)
    if mfcc
    mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(firearm_path,file)
    mfcc_flat=flattenArray(mfcc,printtxt=True)
    
    mfcc_centered=centerFeature(mfcc_flat)
    log_ms_centered=centerFeature(flattenArray(log_melspec))
    log_h_centered=centerFeature(flattenArray(log_harmonic))
    log_p_centered=centerFeature(flattenArray(log_percussive))
    
    MFCC_fa=np.concatenate((MFCC_fa,mfcc_centered),axis=0)
    LOG_MS_fa=np.concatenate((LOG_MS_fa,log_ms_centered),axis=0)
    LOG_H_fa=np.concatenate((LOG_H_fa,log_h_centered),axis=0)
    LOG_P_fa=np.concatenate((LOG_P_fa,log_p_centered),axis=0)

/opt/anaconda3/envs/add_librosa/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


The shape of the feature is  (1, 3520)
484036__mattix__rifle-gun-shot-02.wav
The shape of the feature is  (1, 1300)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3520 and the array at index 1 has size 1300

In [13]:
print(MFCC_fa.shape)
print(LOG_MS_fa.shape)
print(LOG_H_fa.shape)
print(LOG_P_fa.shape)

(32, 3520)
(32, 22528)
(32, 22528)
(32, 22528)


In [12]:
np.savetxt("firearm_MFCC_train.csv", MFCC_fa, delimiter=",")
np.savetxt("firearm_logMS_train.csv",LOG_MS_fa,delimiter=",")
np.savetxt("firearm_logH_train.csv",LOG_H_fa,delimiter=",")
np.savetxt("firearm_logP_train.csv",LOG_P_fa,delimiter=",")

## Build the Test Sets

In [13]:
mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(firearm_path,firearm_test_files[0])
MFCC_fa=centerFeature(flattenArray(mfcc))
LOG_MS_fa=centerFeature(flattenArray(log_melspec))
LOG_H_fa=centerFeature(flattenArray(log_harmonic))
LOG_P_fa=centerFeature(flattenArray(log_percussive))

for file in firearm_test_files[1:]:
    mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(firearm_path,file)
    mfcc_flat=flattenArray(mfcc)
    
    mfcc_centered=centerFeature(mfcc_flat)
    log_ms_centered=centerFeature(flattenArray(log_melspec))
    log_h_centered=centerFeature(flattenArray(log_harmonic))
    log_p_centered=centerFeature(flattenArray(log_percussive))
    
    MFCC_fa=np.concatenate((MFCC_fa,mfcc_centered),axis=0)
    LOG_MS_fa=np.concatenate((LOG_MS_fa,log_ms_centered),axis=0)
    LOG_H_fa=np.concatenate((LOG_H_fa,log_h_centered),axis=0)
    LOG_P_fa=np.concatenate((LOG_P_fa,log_p_centered),axis=0)

np.savetxt("firearm_MFCC_test.csv", MFCC_fa, delimiter=",")
np.savetxt("firearm_logMS_test.csv",LOG_MS_fa,delimiter=",")
np.savetxt("firearm_logH_test.csv",LOG_H_fa,delimiter=",")
np.savetxt("firearm_logP_test.csv",LOG_P_fa,delimiter=",")

NameError: name 'obtainFeatures' is not defined

In [ ]:
mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(esc50audio_path,esc50_test_files[0])
MFCC=centerFeature(flattenArray(mfcc))
LOG_MS=centerFeature(flattenArray(log_melspec))
LOG_H=centerFeature(flattenArray(log_harmonic))
LOG_P=centerFeature(flattenArray(log_percussive))

for file in firearm_test_files[1:]:
    mfcc,log_melspec,log_harmonic,log_percussive=obtainFeatures(firearm_path,file)
    mfcc_flat=flattenArray(mfcc)
    
    mfcc_centered=centerFeature(mfcc_flat)
    log_ms_centered=centerFeature(flattenArray(log_melspec))
    log_h_centered=centerFeature(flattenArray(log_harmonic))
    log_p_centered=centerFeature(flattenArray(log_percussive))
    
    MFCC=np.concatenate((MFCC,mfcc_centered),axis=0)
    LOG_MS=np.concatenate((LOG_MS,log_ms_centered),axis=0)
    LOG_H=np.concatenate((LOG_H,log_h_centered),axis=0)
    LOG_P=np.concatenate((LOG_P,log_p_centered),axis=0)
    
np.savetxt("ESC50_MFCC_test.csv", MFCC, delimiter=",")
np.savetxt("ESC50_logMS_test.csv",LOG_MS,delimiter=",")
np.savetxt("ESC50_logH_test.csv",LOG_H,delimiter=",")
np.savetxt("ESC50_logP_test.csv",LOG_P,delimiter=",")